In [1]:
# Longtitude data type transformation
import pandas as pd
# Read Excel file
df = pd.read_excel('C:/Users/90553/Desktop/464/upload/Data_processed.xlsx')

df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

# Maintaining control
print(df['Longitude'].head())

# Save in excel file
df.to_excel('C:/Users/90553/Desktop/data_1.xlsx', index=False)

0    83.866
1    83.866
2    84.590
3    84.805
4    84.603
Name: Longitude, dtype: float64


In [2]:
import pandas as pd

# Read excel file
df = pd.read_excel('C:/Users/90553/Desktop/data_1.xlsx')

# Apply mode imputation for both the first 84 columns and the 102nd column and beyond
for i, column in enumerate(df.columns):
    if i < 84 or i >= 101:  # First 84 columns and 102nd column (index 101) and onwards
        mode_value = df[column].mode()[0]  # Get the mode of the column
        df[column] = df[column].fillna(mode_value)  # Fill missing values ​​with mod

# Save in excel file
df.to_excel('C:/Users/90553/Desktop/data_2.xlsx', index=False)

print("Mode imputation was successfully applied for the first 84 columns and after the 102nd column and the file was saved: 'data_2.xlsx'")

Mode imputation was successfully applied for the first 84 columns and after the 102nd column and the file was saved: 'data_2.xlsx'


In [3]:
import pandas as pd
from sklearn.impute import KNNImputer

# Read Excel file
df = pd.read_excel('C:/Users/90553/Desktop/data_2.xlsx')

# Identify columns 85-101
columns_85_to_101 = df.columns[84:101]  # Columns 85-101
mode_columns = [96, 97, 101]  # Columns to apply mode imputation
knn_columns = [col for i, col in enumerate(columns_85_to_101) if i + 85 not in mode_columns]  # Other columns (to be applied KNN)

# Mode imputation (columns 96, 97, 101)
for column in df.columns[mode_columns]:
    mode_value = df[column].mode()[0]
    df[column] = df[column].fillna(mode_value)

# KNN imputation (including remaining columns, 98 and 99)
knn_imputer = KNNImputer(n_neighbors=5)
df_knn = df[knn_columns]  # Only columns to which KNN will be applied
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=knn_columns)

# Add updated columns to original DataFrame
df.update(df_knn_imputed)

# Save updated file
df.to_excel('C:/Users/90553/Desktop/data_3.xlsx', index=False)

print("Mode and KNN imputation were applied successfully and the file was saved: 'data_3.xlsx'")

Mode and KNN imputation were applied successfully and the file was saved: 'data_3.xlsx'


In [4]:
import pandas as pd
from sklearn.ensemble import IsolationForest

# Read Excel file
df = pd.read_excel('C:/Users/90553/Desktop/data_3.xlsx')

# Select only numeric columns
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Create the Isolation Forest model
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_forest.fit(numeric_df)

# Calculate Outlier scores
df['outlier_score'] = iso_forest.decision_function(numeric_df)

# Set Outlier tags (-1: Outlier, 1: Normal)
df['outlier_label'] = iso_forest.predict(numeric_df)

# Set Outlier score threshold (example: -0.1)
threshold = -0.0184

# Detect rows that are outliers based on score threshold value
df_outliers = df[df['outlier_score'] < threshold]

# Filter non-outlier rows
df_cleaned = df[df['outlier_score'] >= threshold]

# Save outliers and cleared data
df_outliers.to_excel('C:/Users/90553/Desktop/outliers.xlsx', index=False)
df_cleaned.to_excel('C:/Users/90553/Desktop/data_4.xlsx', index=False)

print(f"Outlier detection completed. {len(df_outliers)} the row was determined as outlier and the cleaned data was saved.")

Outlier detection completed. 75 the row was determined as outlier and the cleaned data was saved.
